In [4]:
import requests as rq
import hmac
import hashlib
import json

In [25]:
with open('/Users/chhan/config.json','r') as f:
  config=json.load(f)
config=config['binance_key']
accessKey=config['access_key']
secretKey=config['secret_key']
futureURL = 'https://fapi.binance.com'
spotURL='https://api.binance.com'
headers = {
    'X-MBX-APIKEY': accessKey
}

In [41]:
def requestData(url,method):
  if method=='get':
    return rq.get(url,headers=headers)
  elif method=='post':
    return rq.post(url,headers=headers)

In [42]:
def getCurrentTime():
  subUrl='/api/v3/time'
  return requestData(spotURL+subUrl,'get').json()['serverTime']
  

In [39]:
def createSignature(message):
  return hmac.new(key=secretKey.encode('utf-8'), msg=message.encode('utf-8'),digestmod=hashlib.sha256).hexdigest()

In [63]:
def getCurrentStatus(mainUrl,subUrl,method):
    message = f'timestamp={getCurrentTime()}'
    url = f'{mainUrl}{subUrl}?{message}&signature={createSignature(message)}'
    return requestData(url,method).json()

In [71]:
def getCurrentSpotAccount():
    return getCurrentStatus(spotURL,'/sapi/v3/asset/getUserAsset','post')

In [75]:
# get_user_data(futureURL,'/fapi/v2/account')
# get_user_data(spotURL,'/api/v3/account')
# get_user_data(spotURL,'/sapi/v3/asset/getUserAsset')
# getCurrentSpotAccount()
# getCurrentSpotAccount()
getCurrentStatus(futureURL,'/fapi/v2/account','get')

{'feeTier': 0,
 'canTrade': True,
 'canDeposit': True,
 'canWithdraw': True,
 'tradeGroupId': -1,
 'updateTime': 0,
 'multiAssetsMargin': True,
 'totalInitialMargin': '0.00000000',
 'totalMaintMargin': '0.00000000',
 'totalWalletBalance': '10.00237730',
 'totalUnrealizedProfit': '0.00000000',
 'totalMarginBalance': '10.00237730',
 'totalPositionInitialMargin': '0.00000000',
 'totalOpenOrderInitialMargin': '0.00000000',
 'totalCrossWalletBalance': '10.00237730',
 'totalCrossUnPnl': '0.00000000',
 'availableBalance': '10.00237730',
 'maxWithdrawAmount': '10.00237730',
 'assets': [{'asset': 'FDUSD',
   'walletBalance': '0.00000000',
   'unrealizedProfit': '0.00000000',
   'marginBalance': '0.00000000',
   'maintMargin': '0.00000000',
   'initialMargin': '0.00000000',
   'positionInitialMargin': '0.00000000',
   'openOrderInitialMargin': '0.00000000',
   'maxWithdrawAmount': '0.00000000',
   'crossWalletBalance': '0.00000000',
   'crossUnPnl': '0.00000000',
   'availableBalance': '9.865543